In [1]:
import nilearn
from nilearn import image, datasets, plotting, masking
from pathlib import Path
import pandas as pd
import numpy as np
#import meld
from matplotlib import pyplot as plt
#from numba import jit
from pathlib import Path
import subprocess
import os
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 500)
def run(command, env={}, shell=False):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.Popen(command, stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT, shell=shell,
                               env=merged_env)
    while True:
        line = process.stdout.readline()
        line = str(line, 'utf-8')[:-1]
        print(line)
        if line == '' and process.poll() is not None:
            break
    if process.returncode != 0:
        raise Exception("Non zero return code: %d"%process.returncode)

In [3]:
pd.read_csv('../data/voice/task-passivelistening_events.tsv', sep = '\t')

,onset,duration,trial_type,stimuli
0,12,8,nonvocal,sourcedata/stimuli/non_vocal_01
1,22,8,vocal,sourcedata/stimuli/vocal_01
2,32,8,nonvocal,sourcedata/stimuli/non_vocal_02
3,52,8,nonvocal,sourcedata/stimuli/non_vocal_03
4,62,8,vocal,sourcedata/stimuli/vocal_02
5,82,8,vocal,sourcedata/stimuli/vocal_03
6,102,8,nonvocal,sourcedata/stimuli/non_vocal_04
7,112,8,vocal,sourcedata/stimuli/vocal_04
8,122,8,nonvocal,sourcedata/stimuli/non_vocal_05
9,132,8,vocal,sourcedata/stimuli/vocal_05


In [4]:
base_dir = Path('../data/voice/')
proc_dir = base_dir/'derivatives/afni_proc2'

In [9]:
!which 3dAFNItoNIFTI

/usr/local/apps/afni/current/linux_openmp_64/3dAFNItoNIFTI


In [20]:
#convert REMLs into nifits
bad_list = ['sub-007', 'sub-023']
for sub_dir in sorted(proc_dir.glob('sub-*')):
    sub = sub_dir.parts[-1]
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        stb_dir = sub_dir/'passivelistening/stb/'
        try:
            if not stb_dir.exists():
                stb_dir.mkdir()
                stim_type = 'non_vocal'
                for ii in range(1,41,2):
                    reml_path = sub_dir/('passivelistening/stats.{subn}_REML+tlrc.HEAD[{ii}]'.format(subn = subn, ii = ii))
                    out_path = stb_dir/'REML_{stim_type}_{n}.nii.gz'.format(stim_type = stim_type, n = int((ii-1)/2 + 1))
                    cmd = '3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = out_path, in1 = reml_path)
                    run(cmd, shell = True)
                stim_type = 'vocal'
                for ii in range(42,82,2):
                    reml_path = sub_dir/('passivelistening/stats.{subn}_REML+tlrc.HEAD[{ii}]'.format(subn = subn, ii = ii))
                    out_path = stb_dir/'REML_{stim_type}_{n}.nii.gz'.format(stim_type = stim_type, n = int((ii-42)/2 + 1))
                    cmd = '3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = out_path, in1 = reml_path)
                    run(cmd, shell = True)
        except:
            bad_list.append(sub)

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De

...


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec

In [21]:
bad_list

['sub-007', 'sub-023', 'sub-072', 'sub-077', 'sub-137', 'sub-165']

In [25]:
dat_list = []
for sub_dir in sorted(proc_dir.glob('sub-*')):
    sub = sub_dir.parts[-1]
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        afni_mask_path = sub_dir/('passivelistening/full_mask.{subn}+tlrc.HEAD'.format(subn = subn))
        nii_mask_path = sub_dir/('passivelistening/full_mask.{subn}+tlrc.nii.gz'.format(subn = subn))
        cmd = '3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = nii_mask_path, in1 = afni_mask_path)
        run(cmd,shell = True)

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De

In [ ]:
../data/voice/derivatives/afnie_

In [34]:
cmd = '3dmask_tool -overwrite -prefix ../data/voice/derivatives/afni_proc2/group_full_mask.nii.gz  -frac 0 -dilate_result 2 -2 -inputs ../data/voice/derivatives/afni_proc2/sub-*/passivelistening/full_mask.*.HEAD'
run(cmd, shell = True)

++ processing 216 input datasets...
++ padding all datasets by 0 (for dilations)
++ frac 0 over 216 volumes gives min count 0
++ voxel limits: 0 clipped, 98547 survived, 212749 were zero
++ writing result group_full_mask.nii.gz...
++ Output dataset ../data/voice/derivatives/afni_proc2/group_full_mask.nii.gz



In [25]:
run

PosixPath('/meld_root/work/data/voice/derivatives/afni_proc2/sub-001/passivelistening/stats.001_REML+tlrc.HEAD[39]')